# Workshop 1: [pbatoolkit](https://pypi.org/project/pbatoolkit/)'s FEM module

This first workshop on FEM aims to help the reader:
- explore/analyze FEM quantities/operators
- familiarize themself with our [Physics Based Animation Toolkit](https://github.com/Q-Minh/PhysicsBasedAnimationToolkit)'s Python interface

Start by installing the required dependencies by running `pip install -r requirements.txt` on the command line. FEM relies on having meshed geometries at hand, hence we recommend finding clean surface mesh geometry and then using our [mesh processing scripts](/python/tools/mesh/) to tetrahedralize them, if you don't already have access to meshes. You can download surface meshes on [Thingi10K](https://ten-thousand-models.appspot.com/), [TurboSquid](https://www.turbosquid.com/) or other.

If you are interested in profiling calls to FEM operations, also download the [Tracy profiling server](https://github.com/wolfpld/tracy/releases/tag/v0.10), run `tracy.exe` and connect to this process.

## Loading input mesh geometry

Use [meshio](https://pypi.org/project/meshio/) to load meshes from disk to memory, and render them using [polyscope](https://pypi.org/project/polyscope/).

## Constructing FEM meshes

From the input mesh geometry, construct an FEM mesh (i.e. `pbatoolkit.fem.Mesh`) using `pbatoolkit`. Start by exploring the API of the FEM module of `pbatoolkit`. Then, try constructing meshes of different shape function orders on either of line, triangle, and/or tetrahedral meshes. You can do the same for regular geometry (rectangles and boxes) if you want.

`pbatoolkit` expects input matrix data in column major storage (i.e. `order='F'` in [numpy](https://numpy.org/)) and uses double precision floating point numbers as scalars, and 64-bit signed integers as indices. Vertex positions are generally packed in matrix columns, and similarly for cell indices. This format is generally compatible with other packages' APIs, using a no-copy transpose operation (i.e. a row-major `order='C'` matrix with vertex positions in rows has the same memory layout as a column-major `order='F'` matrix with vertex positions in columns).

Use Python's `help` command on `pbatoolkit`'s mesh instance, and check that you can access its nodes' positions, its elements' nodal indices, its dimensionality, element type, etc.

> Note that the FEM mesh and the geometric mesh do NOT share the same element indices. Both the FEM and geometric mesh have the same number of elements, and they are ordered in exactly the same way, but the geometric mesh's array of vertex positions does not necessarily correspond to the FEM mesh's array of node positions.

Keep an eye on how the number of FEM nodes changes when the mesh's shape function order changes, and the computational cost of constructing higher order meshes. Visualize the FEM mesh's nodes on top of the geometric mesh using `polyscope`.

In [ ]:
# Import modules of interest
import pbatoolkit as pbat
import pbatoolkit.profiling
# Consult FEM module's API
help(pbat.fem)
help(pbat.fem.Mesh)

## Mass matrix

1. Construct the mass matrix using `pbatoolkit`'s built-in `MassMatrix` class, and then using only the shape function and quadrature matrices, and compare the resulting mass matrices. Call `to_matrix` to get a `scipy` sparse matrix from `pbatoolkit`'s mass matrix instance.
2. Analyze its properties. Is it square, symmetric, positive definite? What is its sparsity? 
3. What happens if you use the correct quadrature order, a lower quadrature order, and a higher quadrature order? 
4. How computationally costly is mass matrix construction in both cases?

## Load vector

1. Construct a piecewise polynomial load vector using `pbatoolkit`'s shape function and quadrature matrices (use `pbat.fem.shape_function_matrix` and `pbat.fem.inner_product_weights`). Refer to [examples](/python/examples/) for help. Alternatively, the load vector can be defined at nodes, in which case you only need the mass matrix (`pbat.fem.MassMatrix`). You can use some common forcing function, like gravity, i.e. $f_e(X) = [0, 0, -9.81]$. Compare the resulting discrete load vectors.
2. Try constructing different load vectors using different quadrature orders and analyze the results.
3. How computationally costly is its construction in all cases?

## Laplacian matrix
 1. Construct the Laplacian matrix using `pbatoolkit`'s built-in `pbat.fem.Laplacian` class, and then using only the gradient (`pbat.fem.Gradient`) and quadrature matrices (`pbat.fem.inner_product_weights`), and compare the resulting Laplacians.
 2. Analyze its properties, similar to the mass matrix case.
 3. Try varying the quadrature orders and evaluate the consequences.
 4. What is the computational cost of its different construction approaches?

## Higher order FEM

When using higher-order FEM, it is often useful to visualize our discretized functions on refined versions of our input geometric mesh. This is due to the fact that renderers that support scalar field visualization use linear interpolation to fill the colors between mesh vertices, so we need to use "visual" meshes of higher resolution than our "geometric" mesh to see any difference.

You can refine meshes using our [mesh processing scripts](/python/tools/mesh/). Then, in code, use `pbat.geometry.bvh` to compute a spatial query acceleration data structure, our bounding volume implementation (BVH), on the "geometric" mesh. Vertices of the "visual" mesh should be mapped to the "computational" mesh (i.e. the FEM mesh) by 
1. Querying in which element of the "geometric" mesh they reside. This query can be greatly accelerated using the BVH. 
2. Use `pbat.fem.reference_positions` to compute the reference element positions of the visual mesh nodes. 
3. Use `pbat.fem.shape_functions_at` to evaluate the shape functions at the visual mesh nodes, using their corresponding reference positions. 
4. Evaluate the FEM function by coefficient-wise multiplication between nodal coefficients and associate shape functions evaluated at visual mesh nodes, and compute their sum separately (for each visual mesh node).

Refer to [this higher order example](/python/examples/laplace.py) for help.

## Solutions

Scripts in the [examples](/python/examples/) folder show different ways to construct relevant FEM quantities using `pbatoolkit` for different mesh types, different shape function orders, and different construction approaches.